### Adjust Word Group Repeat With Youtube Link

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from nltk import ngrams
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 5000  # 28 native word end index

# file extention
file_ext = "Threegram"

In [4]:
def df_col_value_join_comma(df, df_columns_list):
    '''
    df_col_value_join_comma(df_test, ["video_id","start_time","end_time"])\n
    function used for selected column value join with comma in one row
    '''
    column_value_list = []
    for column in df_columns_list:
        list_var = df[f"{column}"].to_list()
        list_var_string = [str(x) for x in list_var] 
        list_var_join = ",".join(list_var_string)
        column_value_list.append(list_var_join)

    df_result = pd.DataFrame([[column_value_list[0],column_value_list[1],column_value_list[2]]], columns=df_columns_list)

    return df_result

In [5]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Talk Time 3/Result/3-Adjust Word Group Repeat With Youtube Link"

Path(path).mkdir(parents=True, exist_ok=True)

##### Link Count

In [17]:
path_folder = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Talk Time 3/Result/2-Find Word Group Youtube Link"
file = f"{lang_folder.capitalize()}_{file_ext}_With_{word_end}_Word_Selected_Manuel.xlsx"
sheet = "Sheet1"  # Sheet1

In [18]:
df_link_selected_manuel = pd.read_excel(f"{path_folder}/{file}", sheet_name=f"{sheet}")
df_link_selected_manuel

,search_string,start_time,end_time,video_id
0,ama bu,174,176,1boYNWzDEVQ
1,ama bu,7745,7747,eJdWTp7DDf0
2,ama bu,355,356,ZAlqIYFjm8Y
3,ama bu,205,207,vrqdZwTD5fs
4,ama bu,281,283,ebP0C2MSwgw
...,...,...,...,...
126,seni seviyorum,1891,1893,Yr2ZLfWT0i8
127,yardım et,6663,6665,oCsHNgql560
128,yardım et,2601,2603,05PK5asBrP0
129,yardım et,7734,7737,Ca9zvaqTlXk


In [19]:
df_link_selected_manuel["video_url"] = "https://www.youtube.com/watch?v="+df_link_selected_manuel['video_id'].map(str)+"&t="+df_link_selected_manuel['start_time'].map(str)+"s"
df_link_selected_manuel

,search_string,start_time,end_time,video_id,video_url
0,ama bu,174,176,1boYNWzDEVQ,https://www.youtube.com/watch?v=1boYNWzDEVQ&t=...
1,ama bu,7745,7747,eJdWTp7DDf0,https://www.youtube.com/watch?v=eJdWTp7DDf0&t=...
2,ama bu,355,356,ZAlqIYFjm8Y,https://www.youtube.com/watch?v=ZAlqIYFjm8Y&t=...
3,ama bu,205,207,vrqdZwTD5fs,https://www.youtube.com/watch?v=vrqdZwTD5fs&t=...
4,ama bu,281,283,ebP0C2MSwgw,https://www.youtube.com/watch?v=ebP0C2MSwgw&t=...
...,...,...,...,...,...
126,seni seviyorum,1891,1893,Yr2ZLfWT0i8,https://www.youtube.com/watch?v=Yr2ZLfWT0i8&t=...
127,yardım et,6663,6665,oCsHNgql560,https://www.youtube.com/watch?v=oCsHNgql560&t=...
128,yardım et,2601,2603,05PK5asBrP0,https://www.youtube.com/watch?v=05PK5asBrP0&t=...
129,yardım et,7734,7737,Ca9zvaqTlXk,https://www.youtube.com/watch?v=Ca9zvaqTlXk&t=...


In [23]:
df_count = df_link_selected_manuel.groupby(["search_string"])[["search_string"]].count()
df_count.rename(columns={"search_string":"count"}, inplace=True)
df_count.sort_values(by="count", ascending=False, inplace=True)
df_count.reset_index(inplace=True)
df_count

,search_string,count
0,ama bu,10
1,bu ne,8
2,biraz daha,7
3,bu kadar,7
4,hazır mısın,6
5,ne zaman,6
6,bir tane,6
7,ne diyorsun,6
8,ne olacak,5
9,bir gün,5


In [24]:
df_count = df_count[df_count["count"] > 2]

In [25]:
df_count_merge = pd.merge(df_link_selected_manuel, df_count, how="inner",on= "search_string")
df_count_merge.sort_values(by=["count","search_string"], ascending=False, inplace=True)
df_count_merge.reset_index(drop=True, inplace=True)
df_count_merge

,search_string,start_time,end_time,video_id,video_url,count
0,ama bu,174,176,1boYNWzDEVQ,https://www.youtube.com/watch?v=1boYNWzDEVQ&t=...,10
1,ama bu,7745,7747,eJdWTp7DDf0,https://www.youtube.com/watch?v=eJdWTp7DDf0&t=...,10
2,ama bu,355,356,ZAlqIYFjm8Y,https://www.youtube.com/watch?v=ZAlqIYFjm8Y&t=...,10
3,ama bu,205,207,vrqdZwTD5fs,https://www.youtube.com/watch?v=vrqdZwTD5fs&t=...,10
4,ama bu,281,283,ebP0C2MSwgw,https://www.youtube.com/watch?v=ebP0C2MSwgw&t=...,10
...,...,...,...,...,...,...
120,bu gece,440,442,cm4mNQcOvqg,https://www.youtube.com/watch?v=cm4mNQcOvqg&t=...,3
121,bu gece,1288,1290,WhhUMO5C0SE,https://www.youtube.com/watch?v=WhhUMO5C0SE&t=...,3
122,bir saniye,287,288,BEH6RWDGy7c,https://www.youtube.com/watch?v=BEH6RWDGy7c&t=...,3
123,bir saniye,279,281,7oozYl45nzQ,https://www.youtube.com/watch?v=7oozYl45nzQ&t=...,3


In [26]:
df_count.to_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Talk Time 3/Data/Deployment/Talk Time 3 {file_ext} Video List.xlsx", index=False)

In [22]:
df_count_merge.to_excel(f"{path_folder}/{lang_folder.capitalize()}_{file_ext}_With_{word_end}_Word_Selected_Count_Manuel.xlsx", index=False)

##### Link Repeat

In [22]:
path_folder_file2 = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Talk Time 3/Result/2-Find Word Group Youtube Link/{lang_folder.capitalize()}_{file_ext}_With_{word_end}_Word_Selected_Count_Manuel.xlsx"
sheet2 = "Sheet1"  # Sheet1

In [23]:
df_word_group_time_loc = pd.read_excel(f"{path_folder_file2}", sheet_name=f"{sheet2}")
#df_word_group_time_loc = pd.read_excel(f"Twogram Threegram Youtube Link Selected_Manuel.xlsx")   
df_word_group_time_loc

,search_string,start_time,end_time,video_id,video_url,count
0,ama bu,174,176,1boYNWzDEVQ,https://www.youtube.com/watch?v=1boYNWzDEVQ&t=...,7
1,ama bu,205,207,vrqdZwTD5fs,https://www.youtube.com/watch?v=vrqdZwTD5fs&t=...,7
2,ama bu,281,283,ebP0C2MSwgw,https://www.youtube.com/watch?v=ebP0C2MSwgw&t=...,7
3,ama bu,487,491,kSS3oSQpEbo,https://www.youtube.com/watch?v=kSS3oSQpEbo&t=...,7
4,ama bu,234,236,V3AwyEcjM_0,https://www.youtube.com/watch?v=V3AwyEcjM_0&t=...,7
...,...,...,...,...,...,...
116,bu gece,440,442,cm4mNQcOvqg,https://www.youtube.com/watch?v=cm4mNQcOvqg&t=...,3
117,bu gece,1288,1290,WhhUMO5C0SE,https://www.youtube.com/watch?v=WhhUMO5C0SE&t=...,3
118,bir saniye,287,288,BEH6RWDGy7c,https://www.youtube.com/watch?v=BEH6RWDGy7c&t=...,3
119,bir saniye,279,281,7oozYl45nzQ,https://www.youtube.com/watch?v=7oozYl45nzQ&t=...,3


In [24]:
search_list = df_word_group_time_loc.iloc[:,0].unique()
len(search_list)

26

In [25]:
# other option 
# mUf7VNqChac =>  black screen
# 0_CDMstFg7M => 10sn
# bj1JRuyYeco => 20sn
# cElhIDdGz7M => screensaver
default_video_id = "Q-8I-uMUMYA"
df_link_default = pd.DataFrame(data=[["repeat",5,7,"repeat_again",f"{default_video_id}",f"https://www.youtube.com/watch?v={default_video_id}&t=0s"]], columns=["search_string","start_time","end_time","sentence","video_id","video_url"])
df_link_default

,search_string,start_time,end_time,sentence,video_id,video_url
0,repeat,5,7,repeat_again,Q-8I-uMUMYA,https://www.youtube.com/watch?v=Q-8I-uMUMYA&t=0s


In [26]:
## All word group result convert to join result in one row
#df_result_repeat = pd.DataFrame()
#for word_group in search_list:
#    # for repeat 1
#    df_word_group_search_repeat1 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group]
#    # for repeat 2
#    try:
#        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].sample(1)
#    except:
#        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].tail(1)
#    df_word_group_search_repeat2.reset_index(drop=True, inplace=True)   
#    # repeat block
#    for i in range(len(df_word_group_search_repeat1)):        
#        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat1.iloc[[i,]]], axis=0)
#
#    try:
#        df_link_default_var = df_link_default
#        word_time_diff_var = df_word_group_search_repeat2.loc[0 ,"end_time"] - df_word_group_search_repeat2.loc[0 ,"start_time"]
#        if word_time_diff_var < 4:
#            word_time_diff_var2 = 3
#        else:
#            word_time_diff_var2 = word_time_diff_var+1.0
#
#        df_link_default_var.loc[0,"end_time"] = df_link_default_var.loc[0,"start_time"] + word_time_diff_var2
#    except:
#        pass
#
#    for j in range(3):
#        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat2], axis=0)
#        df_result_repeat = pd.concat([df_result_repeat,df_link_default_var], axis=0)
#
#df_result_repeat.reset_index(drop=True, inplace=True)
#df_result_repeat_join = df_col_value_join_comma(df_result_repeat, ["video_id","start_time","end_time"])
#
#df_result_repeat.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Talk_Time2.xlsx", index=False)
#df_result_repeat_join.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Talk_Time_Join2.xlsx", index=False)

In [27]:
#df_result_repeat

In [28]:
#(df_result_repeat["end_time"].sum()-df_result_repeat["start_time"].sum())/60

In [29]:
# Each word group result convert to join result in one row
df_all_join_result = pd.DataFrame()
seq_num = 0  # option
for word_group in search_list:
    df_result_repeat = pd.DataFrame()
    # for repeat 1
    df_word_group_search_repeat1 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group]
    # for repeat 2
    try:
        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].sample(1)
    except:
        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].tail(1)
    df_word_group_search_repeat2.reset_index(drop=True, inplace=True)   
    # repeat block
    for i in range(len(df_word_group_search_repeat1)):        
        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat1.iloc[[i,]]], axis=0)

    try:
        df_link_default_var = df_link_default
        word_time_diff_var = df_word_group_search_repeat2.loc[0 ,"end_time"] - df_word_group_search_repeat2.loc[0 ,"start_time"]
        if word_time_diff_var < 4:
            word_time_diff_var2 = 3
        else:
            word_time_diff_var2 = word_time_diff_var+1.0

        df_link_default_var.loc[0,"end_time"] = df_link_default_var.loc[0,"start_time"] + word_time_diff_var2
    except:
        pass

    for j in range(3):
        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat2], axis=0)
        df_result_repeat = pd.concat([df_result_repeat,df_link_default_var], axis=0)
        df_result_repeat.reset_index(drop=True, inplace=True)

    df_result_repeat_var = df_col_value_join_comma(df_result_repeat, ["video_id","start_time","end_time"]) 
    df_result_repeat_var.loc[0,["search_string"]] = f"{word_group}"  # option for search string
    df_result_repeat_var.loc[0,["count"]] = df_result_repeat.loc[0,"count"] # option for count num     
    df_all_join_result = pd.concat([df_all_join_result,df_result_repeat_var], axis=0)

df_all_join_result.reset_index(drop=True, inplace=True)
df_all_join_result["web_format"] = f"[play_lms_videos video_id="+df_all_join_result["video_id"].map(str)+" start_time="+df_all_join_result["start_time"].map(str)+" end_time="+df_all_join_result["end_time"].map(str)+"]"
df_all_join_result.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Youtube_Talk_Time_Join_{file_ext}.xlsx", index=False)


In [30]:
df_all_join_result

,video_id,start_time,end_time,search_string,count,web_format
0,"1boYNWzDEVQ,vrqdZwTD5fs,ebP0C2MSwgw,kSS3oSQpEb...","174,205,281,487,234,2056,744,487,5,487,5,487,5","176,207,283,491,236,2057,746,491,10,491,10,491,10",ama bu,7.0,"[play_lms_videos video_id=1boYNWzDEVQ,vrqdZwTD..."
1,"AaoZnt4E9EU,pcAhoVDv2Gs,FTEw92IMSeI,2oY0tvlzuJ...","2110,7017,435,960,2927,6108,6377,960,5,960,5,9...","2112,7018,437,962,2929,6110,6378,962,8,962,8,9...",bu ne,7.0,"[play_lms_videos video_id=AaoZnt4E9EU,pcAhoVDv..."
2,"s4MqASVecXc,EcfG7zdjBYQ,iMj5gB0XHRw,fJ75tL1Oh9...","3656,2929,3867,236,2620,70,3071,3071,5,3071,5,...","3658,2931,3869,238,2622,72,3073,3073,8,3073,8,...",bu kadar,7.0,"[play_lms_videos video_id=s4MqASVecXc,EcfG7zdj..."
3,"SCYnEIq3mCY,FO07ssbT9bA,eHWn_tBs3R0,0fBopIv7q3...","378,1674,990,43,205,132,1746,1746,5,1746,5,1746,5","380,1676,992,44,207,134,1748,1748,8,1748,8,1748,8",biraz daha,7.0,"[play_lms_videos video_id=SCYnEIq3mCY,FO07ssbT..."
4,"LC2QScFT5QI,8xaLfHCU2KM,W263ryKMPH0,otZfpEWXaq...","2505,2537,1926,489,2068,7378,7378,5,7378,5,7378,5","2507,2539,1928,491,2070,7380,7380,8,7380,8,7380,8",ne zaman,6.0,"[play_lms_videos video_id=LC2QScFT5QI,8xaLfHCU..."
5,"x6aEnqpTUBo,YN6xuSz86PA,cslXqrcH5qg,hepgJ44h3W...","1986,4190,5403,2586,2301,5263,2301,5,2301,5,23...","1988,4192,5405,2588,2303,5265,2303,8,2303,8,23...",ne diyorsun,6.0,"[play_lms_videos video_id=x6aEnqpTUBo,YN6xuSz8..."
6,"778wDupqulw,mAlVCfeikh4,gHjUEWlfmOw,VpSpwWMW5e...","280,1761,953,1714,53,1129,1129,5,1129,5,1129,5","282,1763,955,1717,55,1131,1131,8,1131,8,1131,8",hazır mısın,6.0,"[play_lms_videos video_id=778wDupqulw,mAlVCfei..."
7,"iszZyYIJVe8,Q3Azp5eUgNE,nnjPUQlzjlg,ePPcGJzpOy...","224,6168,746,94,308,1957,224,5,224,5,224,5","226,6170,748,96,309,1959,226,8,226,8,226,8",bir tane,6.0,"[play_lms_videos video_id=iszZyYIJVe8,Q3Azp5eU..."
8,"kyWbeG_7T5A,O1pUs5EpWVQ,OE8ujtE5ybI,MWMmHcPKvf...","3263,975,408,3951,297,408,5,408,5,408,5","3265,977,410,3953,299,410,8,410,8,410,8",öyle değil,5.0,"[play_lms_videos video_id=kyWbeG_7T5A,O1pUs5Ep..."
9,"7hf6KFdMcC0,SXvPXzCgIao,aLY7EZoOlVo,2Me6SuIfpS...","2272,158,1781,7013,4119,7013,5,7013,5,7013,5","2274,161,1783,7015,4121,7015,8,7015,8,7015,8",ne olacak,5.0,"[play_lms_videos video_id=7hf6KFdMcC0,SXvPXzCg..."


#### Copy Move And Delete

In [31]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{word_end}_Word_Youtube_Talk_Time_Join_{file_ext}.xlsx")
output_file

['Turkish_5000_Word_Youtube_Talk_Time_Join_Twogram.xlsx']

In [32]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [33]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

In [ ]:
#df_test["web_format"] = f"[play_lms_videos video_id="+df_test["video_id"].map(str)+" start_time="+df_test["start_time"].map(str)+" end_time="+df_test["end_time"].map(str)+"]"
#df_test